# Gallery Example: Er/M/1 Queue (Erlang Arrivals)

This example demonstrates an Er/M/1 queueing system:
- **Arrivals**: Erlang distribution (less variable than Poisson)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Erlang distribution (with shape parameter k=5) has lower variance than exponential, representing more regular arrivals than a Poisson process.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_erlm1():    """Create Er/M/1 queueing model"""    model = Network('Er/M/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Erlang arrival with mean=1 and order=5 (less variable than exponential)    source.set_arrival(oclass, Erlang.fit_mean_and_order(1, 5))    # Exponential service with rate=2 (mean service time = 0.5)    queue.set_service(oclass, Exp(2))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_erlm1()

## Theoretical Analysis for Er/M/1

For Er/M/1 with:
- **Arrival rate**: λ = 1 (Erlang with mean=1, shape=5)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

The Erlang distribution characteristics:
- **Mean**: 1.0
- **Variance**: 1/5 = 0.2 (lower than exponential variance = 1)
- **Coefficient of Variation**: C²ₐ = 1/5 = 0.2

This lower arrival variability typically results in better performance than M/M/1.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare Er/M/1 with M/M/1
print("\n=== Comparison with M/M/1 ===")

# Create equivalent M/M/1 model
def create_mm1_equivalent():
    model_mm1 = Network('M/M/1-Equivalent')
    source = Source(model_mm1, 'Source')
    queue = Queue(model_mm1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_mm1, 'Sink')
    
    oclass = OpenClass(model_mm1, 'Class')
    source.set_arrival(oclass, Exp(1))  # Same mean arrival rate
    queue.set_service(oclass, Exp(2))   # Same service rate
    
    P = model_mm1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

model_mm1 = create_mm1_equivalent()
solver_erlm1 = MVA(model)
solver_mm1 = MVA(model_mm1)

avg_table_erlm1 = solver_erlm1.avg_table()
avg_table_mm1 = solver_mm1.avg_table()

print("Er/M/1 Results:")
print(avg_table_erlm1)

print("\nM/M/1 Results:")
print(avg_table_mm1)

# Extract key metrics for comparison
erlm1_resp = float(avg_table_erlm1.iloc[1, 2])  # Queue response time
erlm1_length = float(avg_table_erlm1.iloc[1, 3])  # Queue length

mm1_resp = float(avg_table_mm1.iloc[1, 2])  # Queue response time
mm1_length = float(avg_table_mm1.iloc[1, 3])  # Queue length

print(f"\nPerformance Comparison:")
print(f"Response Time: Er/M/1={erlm1_resp:.4f}, M/M/1={mm1_resp:.4f}")
print(f"Queue Length: Er/M/1={erlm1_length:.4f}, M/M/1={mm1_length:.4f}")
print(f"Improvement: {((mm1_resp - erlm1_resp) / mm1_resp * 100):.1f}% better response time with Erlang arrivals")

In [ ]:
# Analyze impact of Erlang order parameter
print("\n=== Impact of Erlang Order Parameter ===")

def create_erlang_model(order):
    """Create Er/M/1 model with specified Erlang order"""
    model_erl = Network(f'Er{order}/M/1')
    source = Source(model_erl, 'Source')
    queue = Queue(model_erl, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_erl, 'Sink')
    
    oclass = OpenClass(model_erl, 'Class')
    source.set_arrival(oclass, Erlang.fit_mean_and_order(1, order))
    queue.set_service(oclass, Exp(2))
    
    P = model_erl.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl

# Test different Erlang orders
orders = [1, 2, 3, 5, 10]  # Order 1 = Exponential

print("Order | CV² | Response Time | Queue Length")
print("-" * 45)

for order in orders:
    model_order = create_erlang_model(order)
    solver = MVA(model_order)
    avg_table = solver.avg_table()
    
    resp_time = float(avg_table.iloc[1, 2])
    queue_length = float(avg_table.iloc[1, 3])
    cv_squared = 1.0 / order  # Coefficient of variation squared for Erlang
    
    print(f"  {order:2d}  | {cv_squared:.3f} |     {resp_time:.4f}    |    {queue_length:.4f}")

print("\nNote: Higher order (lower CV²) leads to better performance due to reduced arrival variability.")